In [ ]:
# import any libraries you need here
!pip install pandas scikit-learn tensorflow transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

Data format - Prompt, Response, Model in a CSV file

Data loading below

In [ ]:
df = pd.read_csv('/content/584 midterm project - Sheet1.csv')
df.head()

,Prompt,Response,Model
0,Yesterday I went,to the beach and watched the sunset.,ChatGPT-4o
1,Yesterday I went,for a long walk in the park and cleared my mind.,ChatGPT-4o
2,Yesterday I went,shopping for new books and lost track of time.,ChatGPT-4o
3,Yesterday I went,to a concert with friends and had an amazing t...,ChatGPT-4o
4,Yesterday I went,to the gym and felt energized after my workout.,ChatGPT-4o


In [ ]:
print(df.columns)

Index(['Prompt', 'Response', 'Model'], dtype='object')


In [ ]:
# Checking for null vals in dataset
print(df.isnull().sum())

Prompt      0
Response    0
Model       0
dtype: int64


In [ ]:
# If null vals present, run this
df.dropna(inplace=True)
# Fill NaN values with empty strings in the 'prompt' and 'response' columns
df['Prompt'].fillna('', inplace=True)
df['Response'].fillna('', inplace=True)


In [ ]:
# Combine propt and response into a single feature
df['text'] = df['Prompt'] + " " + df['Response']

# Encode the model names (target variable)
le = LabelEncoder()
df['model_encoded'] = le.fit_transform(df['Model'])

# Features and target var
x = df['text']
y = df['model_encoded']

# Preparing train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Using Logistic Regression for classification

In [ ]:
# Create a TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the data
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [ ]:
# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(x_train_tfidf, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

In [ ]:
# If using logistic regression:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.525
              precision    recall  f1-score   support

  ChatGPT-4o       0.41      0.32      0.36        37
      Claude       0.56      0.35      0.43        43
      Cohere       0.66      0.69      0.67        45
      Gemini       0.46      0.74      0.57        35

    accuracy                           0.53       160
   macro avg       0.52      0.53      0.51       160
weighted avg       0.53      0.53      0.51       160



Using BERT for classification

In [ ]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode text data
x_train_tokens = tokenizer(list(x_train), padding=True, truncation=True, return_tensors='tf', max_length=128)
x_test_tokens = tokenizer(list(x_test), padding=True, truncation=True, return_tensors='tf', max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load a pre-trained transformer model for sequence classification
bert_model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))

# Define input tensors and labels for training
train_dataset = tf.data.Dataset.from_tensor_slices((dict(x_train_tokens), y_train)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(x_test_tokens), y_test)).batch(16)

# Compile the model
bert_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

# Train the model
bert_model.fit(train_dataset, epochs=100)

# Evaluate on the test set
bert_model.evaluate(test_dataset)


model.safetensors:  74%|#######3  | 325M/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100
40/40 [==============================] - 51s 174ms/step - loss: 0.9834 - accuracy: 0.5266
Epoch 2/100
40/40 [==============================] - 6s 150ms/step - loss: 0.5259 - accuracy: 0.7437
Epoch 3/100
40/40 [==============================] - 6s 157ms/step - loss: 0.3411 - accuracy: 0.8484
Epoch 4/100
40/40 [==============================] - 6s 151ms/step - loss: 0.2080 - accuracy: 0.9141
Epoch 5/100
40/40 [==============================] - 6s 158ms/step - loss: 0.1394 - accuracy: 0.9547
Epoch 6/100
40/40 [==============================] - 6s 153ms/step - loss: 0.1453 - accuracy: 0.9500
Epoch 7/100
40/40 [==============================] - 6s 160ms/step - loss: 0.1251 - accuracy: 0.9609
Epoch 8/100
40/40 [==============================] - 6s 153ms/step - loss: 0.0800 - accuracy: 0.9688
Epoch 9/100
40/40 [==============================] - 6s 161ms/step - loss: 0.0823 - accuracy: 0.9703
Epoch 10/100
40/40 [==============================] - 6s 155ms/step - loss: 0.0671 - accur

[1.2984601259231567, 0.831250011920929]

In [ ]:
predictions = bert_model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Accuracy and report
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, predicted_labels, target_names=le.classes_))

10/10 [==============================] - 5s 68ms/step
Accuracy: 0.825
              precision    recall  f1-score   support

  ChatGPT-4o       0.91      0.84      0.87        37
      Claude       0.85      0.67      0.75        43
      Cohere       0.88      0.93      0.90        45
      Gemini       0.68      0.86      0.76        35

    accuracy                           0.82       160
   macro avg       0.83      0.83      0.82       160
weighted avg       0.84      0.82      0.82       160



In [ ]:
# Save transformer model
bert_model.save_pretrained('bert_text_classifier')
tokenizer.save_pretrained('bert_text_classifier')

('bert_text_classifier/tokenizer_config.json',
 'bert_text_classifier/special_tokens_map.json',
 'bert_text_classifier/vocab.txt',
 'bert_text_classifier/added_tokens.json',
 'bert_text_classifier/tokenizer.json')

In [ ]:
# Zipping model to export

!zip -r "/content/text_classifier_model.zip" "/content/bert_text_classifier"

  adding: content/bert_text_classifier/ (stored 0%)
  adding: content/bert_text_classifier/tf_model.h5 (deflated 8%)
  adding: content/bert_text_classifier/special_tokens_map.json (deflated 42%)
  adding: content/bert_text_classifier/tokenizer.json (deflated 71%)
  adding: content/bert_text_classifier/tokenizer_config.json (deflated 76%)
  adding: content/bert_text_classifier/vocab.txt (deflated 53%)
  adding: content/bert_text_classifier/config.json (deflated 51%)


Loading and using model later

In [ ]:
import numpy as np

# Load transformer model
loaded_bert_model = TFAutoModelForSequenceClassification.from_pretrained('bert_text_classifier')
loaded_tokenizer = AutoTokenizer.from_pretrained('bert_text_classifier')

# For new predictions:
new_text = ["I spoke to my friend about a personal challenge I was facing, and their support and advice provided me with a new sense of clarity and direction."]

new_tokens = loaded_tokenizer(new_text, padding=True, truncation=True, return_tensors='tf', max_length=128)
predictions = loaded_bert_model.predict(dict(new_tokens))

# Get the index of the class with the highest logit (predicted class)
predicted_class_idx = np.argmax(predictions.logits, axis=1)

# Decode the predicted class index to get the original label (model name)
predicted_model_name = le.inverse_transform(predicted_class_idx)

print(f"Predicted model: {predicted_model_name[0]}")

Some layers from the model checkpoint at bert_text_classifier were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bert_text_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


1/1 [==============================] - 5s 5s/step
Predicted model: Cohere
